# 0-2. Estimator

### Estimator란?
  - 최근까지 텐서플로우 팀에서 적극적으로 지원하던 고수준 API (지향점)
  - Slim 및 Keras 등의 Contrib의 좋은 라이브러리들을 융합 / 발전
  - 실제 서빙 모델에 가장 가까우며, 분산처리 설정 등에 있어 용이
  - tensorflow.dataset.Dataset, tensorflow.feature_column 등의 API는 모두 Estimator와 인테그레이팅 되기 위해 만들어진 API
  
### Estimator의 사용자 입장에서의 장점
  - 커스터마이징이 용이
  - 텐서플로우와 타이트하게 인테그레이팅 되어있음
  - 사실상 현재까지 완성도 가장 높은 고수준 API
  
### Estimator의 사용자 입장에서의 단점
  - 입문단계에서 이용하기가 어려운 편 (다양한 API를 함께 써야 함)
  
### Estimator을 쓰는 방법
  1. premade estimator
  2. custom estimator

### Premade Estimator란?
  - Estimator모듈 내에 빌트인 된 에스티메이터
  - DNN기반 회귀/분류, Boosted Tree등 존재
  - 이외에 공식 릴리즈되지는 않았지만, RNN, GAN등에 대해서도 contrib에 커스텀 에스티메이터 형태로 빌트인
  - 이용자는 해당 에스티메이터 객체를 생성한 뒤, 메서드를 통해 간단하게 학습 및 평가, 추론 진행 가능
  - 원하는 Premade Estimator가 없을 경우 직접 에스티메이터 모델 만들어야

In [ ]:
import tensorflow as tf
import numpy as np

set_vervosity 설정하여 텐서플로우 로그를 받아보는 것을 추천

estimator을 이용할 때 뿐 아니라, 네이티브 텐서플로우를 이용할 때도 마찬가지

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

### 0-2-1. input_fn 정의
  - tensorflow.data.Dataset을 이용해서 정의하는 방법
  - numpy_input_fn, pandas_input_fn을 이용해서 정의하는 방법

In [ ]:
def data_input_fn(features, labels, is_training, batch_size):
    if is_training == True:
        count = None
    else:
        count = 1
    dataset = tf.data.Dataset.from_tensor_slices(({'image':features}, labels.astype(np.int32)))
    return dataset.shuffle(features.shape[0]).repeat(count=count).batch(batch_size)

### 0-2-2. feature_column 정의
  - 데이터 피처의 특성에 맞는 column 인스턴스 정의

In [ ]:
x_col = tf.feature_column.numeric_column("image", shape=[28, 28])

### 0-2-3. Premade estimator 객체 생성
  - hidden_units : DNN내부 Dense layer의 output node들을 리스트 형태로 입력
  - feature_columns : 학습에 이용할 컬럼 정의
  - model_dir : 체크포인트를 저장 및 불러오기할 디렉토리 지정
  - n_classes : 출력층 클래스 개수
  - optimizer : 최적화 방법
  - dropout : dropout 진행할 비율 설정
  
[batch_size, 28, 28] $\rightarrow$ [batch_size, 784]

$\rightarrow$ Dense(784, 512) $\rightarrow$ relu $\rightarrow$ [batch_size, 512]

$\rightarrow$ Dropout $\rightarrow$ Dense(512, 128) $\rightarrow$ relu $\rightarrow$ [batch_size, 128] 

$\rightarrow$ Dropout $\rightarrow$ Dense(128, 10) $\rightarrow$ softmax $\rightarrow$ [batch_size, 10]

In [ ]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[512, 128],
    feature_columns=[x_col],
    model_dir="../generated_output/MNIST_ESTIMATOR/",
    n_classes=10,
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3),
    dropout=0.1,
)

### 0-2-4. 데이터 추출

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.

### 0-2-5. 학습
  - input_fn : 함수의 형태여야 하는 동시에 인풋을 받아야 하므로 람다 함수 이용
  - hooks : tensorflow.train.SessionRunHook, SummarySaverHook등의 객체 이용, 트레이닝 중 특정 시점에 접근하여 훅 실행
  - steps : 학습 반복할 횟수 (epoch 아님에 주의)
  - max_steps : 학습 도달시 종료할 스텝
  - saving_listners : tensorflow.train.CheckpointSaverListener객체 이용, 체크포인트 저장 전후 액션 지정

In [ ]:
estimator.train(
    input_fn=lambda:data_input_fn(x_train, y_train, is_training=True, batch_size=100), 
    hooks=None,
    steps=3000,
    max_steps=None,
    saving_listeners=None
)

### 0-2-6. 평가
  - 학습과정과 유사

In [ ]:
eval_metric = estimator.evaluate(input_fn=lambda:data_input_fn(x_test, y_test, is_training=False, batch_size=128))
print(eval_metric)